In [37]:
import sys
sys.path.append('/Users/shuyangdu/Desktop/ZillowChallenge/zillow-kaggle-challenge')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from data_process.column_schema import (PROPERTIES_RENAME_DICT, TRANSACTION_RENAME_DICT, 
                                        NUMERICAL_COLS, CATEGORICAL_COLS, LOG_COLS)
from data_process.data_process_pipeline import DataProcessPipeline
from data_process.data_transform_processor import DataProcessor
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_all = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged.csv')

In [3]:
data_pipeline = DataProcessPipeline(encode_mode='dummy', use_scale=True)

In [4]:
df = data_pipeline.pre_process(df_all)

In [5]:
iterator = data_pipeline.k_fold(df, 52)

In [6]:
df_train, df_val = iterator.next()

In [7]:
df_train = data_pipeline.post_process(df_train, is_train=True)
df_val = data_pipeline.post_process(df_val, is_train=False)

In [8]:
df_train.shape

(72220, 98)

In [38]:
model = Ridge()

In [10]:
X_train = df_train[data_pipeline.final_feature_cols].values
y_train = df_train[data_pipeline.label_col].values
X_val = df_val[data_pipeline.final_feature_cols].values
y_val = df_val[data_pipeline.label_col].values

# Use new data processor

In [18]:
X_all = DataProcessor.pre_process(df_all)

In [19]:
y_all = df_all['logerror'].values

In [20]:
data_processor_dummy = DataProcessor(use_dummy=True, use_scale=True)

In [22]:
train_idx, val_idx = DataProcessor.k_fold().next()

In [23]:
X_train, X_val = X_all[train_idx], X_all[val_idx]
y_train, y_val = y_all[train_idx], y_all[val_idx]

In [24]:
X_train = data_processor_dummy.fit_transform(X_train)
X_val = data_processor_dummy.transform(X_val)

In [39]:
model.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [40]:
y_pred = model.predict(X_val)

In [41]:
len(np.unique(y_pred))

18054

In [28]:
len(y_pred)

18055

In [42]:
mean_absolute_error(y_val, y_pred)

0.067555232257503248